In [ ]:
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
from pycocotools.coco import COCO
import random
from PIL import Image
import re

%matplotlib inline

# Slicing and Saving

In [ ]:
#load mask
cocoFile = r"" #path to json
coco = COCO(cocoFile)

In [ ]:
numbers = re.compile(r'(\d+)')

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [ ]:
#load ct scan
imagePath = r"" #path to images 
imageStack = []

jsonPath = cocoFile
maskStack = []

with open(jsonPath) as jsonFile:
    cocoAnns = json.load(jsonFile)
jsonFile.close()

images = cocoAnns['images']
annotations = cocoAnns['annotations']

imageList = [[None]*2 for _ in range(len(images))]
for im in range(len(images)):
    fileName = images[im]['file_name']
    index = images[im]['id']
    imageList[im][0] = fileName
    imageList[im][1] = index
    
imageList = np.array(imageList)

for imPath in sorted(glob.glob(imagePath + '/*.png'), key = numericalSort):
   
    basename = os.path.basename(imPath)
    image = cv2.imread(imPath, cv2.IMREAD_GRAYSCALE)
    imageStack.append(image)
    
    if(np.where(imageList[:,0]==basename)[0].size > 0):
        z = np.where(imageList[:,0]==basename)[0][0]
        imId= int(imageList[z,1])
    
        anns_ids = coco.getAnnIds(imgIds=imId, catIds=[], iscrowd=None)
        if not(bool(anns_ids)):
            mask = np.zeros([920,920])
        else:    
            mask = np.zeros([920,920])
            anns = coco.loadAnns(anns_ids)
            for i in range(len(anns)):
                mask += coco.annToMask(anns[i])
            
        maskStack.append(mask)
        
    else:
        mask = np.zeros([920,920])
        maskStack.append(mask)
        
imageStack = np.array(imageStack)
maskStack = np.array(maskStack)
maskStack[maskStack>0] = 1

imageStack = imageStack.transpose(1,2,0)
maskStack = maskStack.transpose(1,2,0)
print('data shape:', np.array(imageStack).shape)
print('mask shape:', np.array(maskStack).shape)

In [ ]:
# Choosing slicing directions for the 3D images (CT images)
SLICE_X = True
SLICE_Y = True
SLICE_Z = True

In [ ]:
SLICE_DECIMATE_IDENTIFIER = 3

In [ ]:
#check max min pixel values of the ct scan
np.min(imageStack), np.max(imageStack), imageStack.shape, type(imageStack)
np.min(maskStack), np.max(maskStack), imageStack.shape, type(maskStack)

In [ ]:
#check max min pixel values of the mask
np.min(maskStack), np.max(maskStack), imageStack.shape, type(maskStack)

In [ ]:
# Showing image slice
imgSlice = imageStack[:,:,1900]
plt.imshow(imgSlice, cmap='gray')
plt.figure(figsize=(10, 10))
plt.show()

In [ ]:
# Showing mask slice
maskSlice = maskStack[:,:,1900]
plt.imshow(maskSlice, cmap='gray')
plt.figure(figsize=(10, 10))
plt.show()

In [ ]:
# Saving volume slices to file
def saveSlice (img, fname, path):
    #img = np.uint8(img * 255) #comment out this line for images 
    fout = os.path.join(path, f'{fname}.png')
    cv2.imwrite(fout, img)
    print(f'[+] Slice saved: {fout}', end='\r')

In [ ]:
# Slicing image in all directions and save
def sliceAndSaveVolumeImage(vol, fname, path):
    (dimx, dimy, dimz) = vol.shape
    print(dimx, dimy, dimz)
    cnt = 0
    if SLICE_X:
        cnt += dimx
        print('Slicing X: ')
        for i in range(dimx):
            saveSlice(vol[i,:,:], fname+f'_slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', path)
            
    if SLICE_Y:
        cnt += dimy
        print('Slicing Y: ')
        for i in range(dimy):
            saveSlice(vol[:,i,:], fname+f'_slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', path)
            
    if SLICE_Z:
        cnt += dimz
        print('Slicing Z: ')
        for i in range(dimz):
            saveSlice(vol[:,:,i], fname+f'_slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', path)
    return cnt


In [ ]:
#slice and save the images
path = r"" #path to sliced images
sliceAndSaveVolumeImage(imageStack, "", path) #scan name 

In [ ]:
#slice and save the masks
path = r"" #path to sliced masks 
sliceAndSaveVolumeImage(maskStack, "", path) #scan name

# Make json

In [ ]:
CATEGORY_TEMPLATE = {
    "id": 1,
    "name": "person",
    "supercategory": None,
    "metadata": {},
    "color": "#69db90",
}

IMAGES_TEMPLATE = {
    "id": 0,
    "width": 530,
    "height": 301,
    "file_name": "",
    "path": "",
    "license": None,
    "fickr_url": None,
    "coco_url": None,
    "date_captured": None,
    "metadata": {},
}

ANNOTATIONS_TEMPLATE = {
    "id": 1,
    "image_id": 0,
    "category_id": 1,
    "width": 530,
    "height": 301,
    "area": 59767,
    "segmentation": [[221, 7, 220, 8, 215, 8]],
    "bbox": [150, 7, 253, 286],
    "metadata": {},
    "color": "#bc07ae",
    "iscrowd": 0,
}

In [ ]:
def mask_to_contour(mask):
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    return contours

def get_random_color():
    return ["#" + "".join([random.choice("ABCDEF0123456789") for _ in range(6)])][
        0
    ].lower()

class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    

In [ ]:
categories = []

category = dotdict(CATEGORY_TEMPLATE)
category.id = 1
category.name = 'root'
category.color = get_random_color()

categories += [category]

annotations = []
images = []
path = r"\*.png" #path to sliced masks
im_num = 1
ann_num = 1 

for img in glob.glob(path):
    im = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    im = im.astype(np.uint8) * 255
    
    image = dotdict(IMAGES_TEMPLATE)
    image.width = im.shape[1]
    image.height = im.shape[0]
    image.file_name = os.path.basename(img)
    image.id = im_num
    images += [image]
    
    contours = mask_to_contour(im)
    
    for contour in contours:
        
        bbox = cv2.boundingRect(contour)
        segmentation = np.array(contour).flatten().tolist()
        
        
        if (len(segmentation) == 4):
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            print(segmentation)
        
        if (len(segmentation) < 4):
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            print(segmentation)
            
        
        annotation = dotdict(ANNOTATIONS_TEMPLATE)
        annotation.id = ann_num
        annotation.image_id = im_num
        annotation.category_id = 1
        annotation.width = bbox[2]
        annotation.height = bbox[3]
        annotation.area = cv2.contourArea(contour)
        annotation.segmentation = [segmentation]
        annotation.bbox = [bbox[0],bbox[1],bbox[2],bbox[3]]
        annotation.color = get_random_color()
        
        annotations += [annotation]
        
        ann_num = ann_num + 1
    
    im_num = im_num + 1 
    
coco = {"annotations": annotations, "categories": categories, "images": images}

In [ ]:
with open(r"", "w") as outfile: #path to new json file containing 3view ground truth
    json.dump(coco, outfile)
outfile.close()

In [ ]:
#inspect to make sure everything looks okay :D

coco_path = r"" #path to new json file containing 3view ground truth
coco = COCO(annotation_file=coco_path)
cat_ids = coco.getCatIds()

print(f"Number of Unique Categories: {len(cat_ids)}")
print("Category IDs:")
print(cat_ids)
cats = coco.loadCats(cat_ids)
cat_names = [cat["name"] for cat in cats]
print("Categories Names:")
print(cat_names)

In [ ]:
img_ids = coco.getImgIds(catIds=1)
print(f"Number of Images Containing root: {len(img_ids)}")

In [ ]:
img_id = img_ids[3600] #choose image ID to view
img_info = coco.loadImgs([img_id])[0]
img_file_name = img_info["file_name"]

print(
    f"Image ID: {img_id}, File Name: {img_file_name}"
)

ann_ids = coco.getAnnIds(imgIds= [img_id], catIds = [1], iscrowd=None)
print(ann_ids)
anns = coco.loadAnns(ann_ids)
print(f"Annotations for Image ID {img_id}:")
print(anns)

In [ ]:
im_path = os.path.join(r"",img_file_name) #path to sliced images 
im = Image.open(im_path)
plt.figure(figsize=(20, 20))
plt.imshow(np.asarray(im), cmap='gray')
coco.showAnns(anns)